In [229]:
# librería para manejar las flexiones gramaticales en el idioma inglés.
!pip install inflect
!pip install --disable-pip-version-check --progress-bar off -q https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

You should consider upgrading via the '/Users/andres/Dropbox/2022-02/BI/proj1-2/Lab4_GR19-main/venv/bin/python -m pip install --upgrade pip' command.
ERROR: Cannot uninstall matplotlib 3.6.2, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps matplotlib==3.6.2'.


In [230]:
seed = 161

import pandas as pd
import warnings; warnings.simplefilter('ignore')
import nltk

from nltk.corpus import stopwords


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


from sklearn.pipeline import Pipeline

from joblib import dump, load


In [231]:
# Descargando las stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1122)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1122)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1122)>


In [232]:
from sklearn.utils import resample



In [233]:
# Se cargan los datos. 

file_name = 'reviews_train_val.csv'
reviews_df = pd.read_csv('data/reviews_train_val.csv', sep = ',')

neg_class_resampled = resample(
    reviews_df, replace=False, n_samples=2000,
    random_state=1234,
)
reviews_df = neg_class_resampled.reset_index(drop=True)

In [234]:
# Se filtran los que no tienen etiqueta
reviews_labeled_df = reviews_df.loc[reviews_df['stars'].notnull()]
reviews_labeled_df.shape

(2000, 2)

In [235]:


X_train, X_test, y_train, y_test = train_test_split(reviews_labeled_df['text'], reviews_labeled_df['stars'], test_size = 0.2, stratify = reviews_labeled_df['stars'], random_state = 1)

X_train.shape

(1600,)

In [236]:
from Processor.TextProcessor import TextProcesser


estimators = [
    ("processer", TextProcesser()),
    ("vect", CountVectorizer(max_df = 0.5, min_df = 0.001)),
]
processing_pipeline = Pipeline(estimators)


In [237]:
X_train = processing_pipeline.fit_transform(X_train).toarray()
X_test = processing_pipeline.transform(X_test).toarray()

In [238]:
dump(processing_pipeline, "assets/pipeline.joblib")

['assets/pipeline.joblib']

In [239]:
from sklearn.linear_model import LogisticRegression

best_model = LogisticRegression(C=50,
 multi_class = 'auto',
 penalty= 'l2',
 solver= 'newton-cg',
 )


In [240]:
best_model.fit(X_train, y_train)

LogisticRegression(C=50, solver='newton-cg')

In [241]:

dump(best_model, "assets/model.joblib")

['assets/model.joblib']

In [242]:
preds_train = best_model.predict(X_train)
preds_test = best_model.predict(X_test)

In [246]:
from sklearn.metrics import precision_recall_fscore_support


def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred)

    avg = list(precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum()
    avg[-1] = total

    class_report_df['avg'] = avg

    return class_report_df.T

df_class_report = pandas_classification_report(y_test, preds_test)

df_class_report.to_json("assets/report.json")


In [247]:
vocabulary = processing_pipeline.steps[1][1].vocabulary_

coef = pd.DataFrame(best_model.coef_)
coef.columns = vocabulary

coef.to_json("assets/coefficients.json")


In [245]:
coef.head()


,cal,appoint,show,job,nee,don,took,12,week,return,...,zen,antique,21st,drag,generously,potatoe,ribey,ribeye,remak,1215
0,0.499477,-0.124021,0.529845,0.019718,-0.110214,-0.097935,0.085161,-0.013951,0.052406,-0.021920,...,-0.005245,-0.014395,-0.138436,-0.015359,-0.021067,0.023913,-0.014337,-0.013435,4.986343e-02,4.986343e-02
1,-0.371662,-0.260570,-0.019277,-0.165760,0.148347,0.119264,-0.082049,-0.083320,-0.010412,0.037685,...,-0.003948,-0.018308,-0.159143,-0.031265,-0.075083,0.141800,0.067787,0.067571,-2.044470e-02,-2.044470e-02
2,-0.477260,-0.000292,-0.239515,-0.052296,-0.020075,-0.064162,-0.164466,0.157979,-0.000214,0.089819,...,-0.139771,-0.078315,-0.141546,0.175741,-0.218283,-0.046317,-0.027091,-0.027093,-5.918506e-02,-5.918506e-02
3,0.356953,0.194483,-0.278374,-0.157483,-0.016474,-0.033786,0.231062,-0.047651,-0.000045,-0.056804,...,0.291911,0.243978,0.340496,-0.182295,0.649105,-0.150430,-0.002463,-0.002463,2.976645e-02,2.976645e-02
4,-0.007509,0.190399,0.007322,0.355821,-0.001583,0.076619,-0.069708,-0.013056,-0.041736,-0.048780,...,-0.142946,-0.132961,0.098630,0.053179,-0.334672,0.031034,-0.023896,-0.024580,-1.158704e-07,-1.158704e-07
